<center> <img src = https://play-lh.googleusercontent.com/YpAV7Q-ZJhI5tzFk_wEX-7-x2BydtnCtFTVUrmq0zAO6jLCLA4nNcfem3p_Pyowg9w width='350'>

# <center> Проект SQL: Анализ вакансий из HeadHunter  

In [1]:
import psycopg2 # библиотека для работы phyton с СУБД PostgreSQL
import pandas as pd

## Предварительный анализ данных

#### количество вакансий

In [3]:
# Используем БД - project_sql, схему - public
connection_project = psycopg2.connect(
   dbname='-',
   user='-',
   host='-',
   password='-',
   port= -
)

query_3_1 = f''' 
            select count(*)
            from public.vacancies
            limit 10
            '''
            # считаем кол-во вакансий

In [5]:
exit_3_1 = pd.read_sql_query(query_3_1, connection_project)# делаем запрос к таблице vacancies
display(exit_3_1)

,count
0,49197


#### количество работодателей

In [6]:
query_3_2 = f''' 
            select COUNT(*)
            FROM public.employers 
            '''

In [7]:
exit_3_2 = pd.read_sql_query(query_3_2, connection_project)
display(exit_3_2)

,count
0,23501


#### количество регионов

In [8]:
query_3_3 = f''' 
            SELECT COUNT(*)
            FROM public.areas
            '''

In [9]:
exit_3_3 = pd.read_sql_query(query_3_3, connection_project)
display(exit_3_3)

,count
0,1362


#### количество сфер деятельности

In [10]:
query_3_4 = f''' 
            SELECT COUNT(*)
            FROM public.industries
            '''

In [11]:
exit_3_4 = pd.read_sql_query(query_3_4, connection_project)
display(exit_3_4)

,count
0,294


# Детальный анализ вакансий

#### сколько вакансий в каждом регионе.Выберите пятёрку лидеров по количеству вакансий:

In [12]:
query_4_1 = f''' 
            SELECT
                ar.name AS area,
                COUNT(*) AS cnt
            FROM 
                public.vacancies AS vc -- Для получения названий регионов и городов соединяем таблицы vacancies и areas
                JOIN public.areas AS ar ON vc.area_id = ar.id
            GROUP BY ar.name
            ORDER BY 2 DESC 
            LIMIT 5
            '''

In [13]:
exit_4_1 = pd.read_sql_query(query_4_1, connection_project)
display(exit_4_1)

# Импортируем библиотеку plotly для визуализации результата
import plotly.express as px
plot = px.bar(data_frame = exit_4_1,
              x = 'cnt',
              height = 400,
              width = 700,
              color = 'area',
              orientation = 'h'
              )
plot.show()

,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


Можно сделать вывод, что в этих городах высокие спрос и предложение на работу и,следовательно, заработная плата выше в сравнении с другими городами, поскольку создается хорошая конкуренция среди компаний и работников.

#### Количество вакансий с заполненым хотя бы одним из двух полей с зарплатой

In [14]:
query_4_2 = f'''
            SELECT 
                COUNT(*)
            FROM public.vacancies vc
            WHERE salary_from IS NOT NULL OR salary_to IS NOT NULL 
            
            '''

In [15]:
exit_4_2 = pd.read_sql_query(query_4_2, connection_project)
display(exit_4_2)

,count
0,24073


Примерно в половине всех вакансий не указа зарплатная вилка. Это может говорить о неуверенности работодателей в том, какой именно специалист ему необходим для выполнения тех или иных задач, какие компетенции будет выполнять сотрудник и какую пользу принесет его работа.
Так же можно предположить, что компании ищут кандидатов, знающих цену своей работе и ориентирующихся в рынке.

#### Cредние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого числа.

In [16]:
query_4_3 = f''' 
            SELECT
                AVG(salary_from) AS lower_mean,
                AVG(salary_to) AS upper_mean
            FROM public.vacancies
            '''

In [17]:
exit_4_3 = pd.read_sql_query(query_4_3, connection_project)
display(exit_4_3)

,lower_mean,upper_mean
0,71064.657901,110536.741923


#### количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях.

In [18]:
query_4_4 = f''' 
            SELECT
                schedule,
                employment,
                COUNT(*)
            FROM public.vacancies
            GROUP BY 1,2
            ORDER BY 3 DESC
            limit 5
            '''

In [19]:
exit_4_4 = pd.read_sql_query(query_4_4, connection_project)
display(exit_4_4)

,schedule,employment,count
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940


Заметное большинство вакансий указывается с графиком Полный день и трудоустройством Полная занятость, такой вариант характерен для работы в офисе. Можно предположить, что значительное большинство компаний хотят иметь возможность контролировать работу своих сотрудников, также помогать и направлять их эффективнее оффлайн. Сочетание этих условий попогает добиться хорошей отдачи от работы в виде результата и выстраиванию долгосрочного сотрудничества с работником.

#### Выведем значения поля experience в порядке возрастания количества вакансий, в которых указан данный вариант опыта.

In [20]:
query_4_5 = f''' 
            SELECT
                experience,
                COUNT(*)
            FROM public.vacancies
            GROUP BY 1
            ORDER BY 2
            '''

In [21]:
exit_4_5 = pd.read_sql_query(query_4_5, connection_project)
display(exit_4_5)

,experience,count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


Наиболее востребованы специалисты начинающие и с опытом от 3 до 6 лет. Молодые и начинающие специалисты развиваются в компании и становятся опытными специалистами глубоко понимающими задачи организации. Таких сотрудников нанимают в основном устойчивые на рынке компании, имеющие возможность "растить" новых специалистов и расширяться. Можно предположить что мы имеем дело с разивающимся рынком.
Молдые компании, пока не готовые обучать малоопытных специалистов, ищут сотруднка со средним или большим опытом, что бы быстро получить результат его работы и наладить выполнение недостающей для бизнеса части работы.

# Анализ работодателей

#### Какие работодатели находятся на первом и пятом месте по количеству вакансий

In [22]:
query_5_1 = f''' 
            SELECT 
                empl.name,
                COUNT(*)
            FROM public.employers empl
                JOIN public.vacancies vc ON vc.employer_id = empl.id
            GROUP BY 1
            ORDER BY 2 DESC
            LIMIT 5
            '''

In [23]:
exit_5_1 = pd.read_sql_query(query_5_1, connection_project)
display(exit_5_1)

,name,count
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


Из вывода следует что Яндекс разместил вакансий больше остальных. Следовательно крупная компания нацелена на расширение и развитие бизнеса в разных секторах. Скорее всего Яндекс, в отличии от других компаний, имеет больше представлений разной продукции на рынке, что делает его бренд узнаваемым (в том числе и компаний вошедших в топ-5). Предположу, что Яндекс расширяет бизнес за счет человеческих ресурсов для предоставления услуг, доступных конечному потребителю (физ.лицам).

#### Для каждого региона вывести количество работодателей и вакансий в нём. Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей

In [24]:
query_5_2 = f''' 
            SELECT
                ar.name,
                COUNT(vc.id) vc,
                COUNT(em.id) emp
            FROM public.areas ar
                LEFT JOIN public.vacancies vc ON ar.id = vc.area_id
                LEFT JOIN public.employers em ON ar.id = em.area
            GROUP BY 1
            ORDER BY vc ASC, emp DESC
            '''

In [25]:
exit_5_2 = pd.read_sql_query(query_5_2, connection_project)
display(exit_5_2)

,name,vc,emp
0,Россия,0,410
1,Казахстан,0,207
2,Московская область,0,75
3,Краснодарский край,0,19
4,Ростовская область,0,18
...,...,...,...
1357,Новосибирск,1149438,1149438
1358,Алматы,1364132,1364132
1359,Минск,2354880,2354880
1360,Санкт-Петербург,6320667,6320667


Регион регистрации некоторых работодателей указывается как страна и не содержит вакансий. Возможно это компании, которые не нуждаются в сотрудниках. 

In [26]:
query_5_3 = f''' 
            SELECT 
                em.name,
                COUNT(DISTINCT vc.area_id)
            FROM public.employers em
                JOIN public.vacancies vc ON em.id = vc.employer_id
            GROUP BY 1
            ORDER BY 2 desc
            '''

In [27]:
exit_5_3 = pd.read_sql_query(query_5_3, connection_project)
display(exit_5_3)

,name,count
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
14761,UniSol,1
14762,UNISTORY LLC,1
14763,UNIT6,1
14764,United Distribution,1


Компании с боьшим числом регионов, в которых размещены вакансии, иеют бльшое покрытие по стране и делают свой продукт популярнее. Такой разброс может означать, что компании имеют бизнес в сфере обслуживания. Поэтому для привлечения новых сотрудников подходят многие регионы имеющие потенциал использования услуг населением.

#### подсчёт количества работодателей, у которых не указана сфера деятельности.

In [28]:
query_5_4 = f''' 
            SELECT
                COUNT(*)
            FROM 
                public.employers_industries em_i
                JOIN public.industries ind ON em_i.industry_id = ind.id
                RIGHT JOIN public.employers em ON em_i.employer_id = em.id
            WHERE ind.name IS NULL
            '''

In [29]:
exit_5_4 = pd.read_sql_query(query_5_4, connection_project)
display(exit_5_4)

,count
0,8419


Более 8 тысяч работодателей не указали сферу деятельности. Возможно эту информацию просто не указали или это связано с определенной секретностью. Поскольку понять, чем занимается организация нельзя напрямую, можно проанализировать наличие вакансий у таких работодателей и установить род деятельности. Если у компании отсутствуют и вакансии, то такая запись в данных не принесет пользы.

#### Название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности.

In [30]:
query_5_5 = f''' 
            SELECT
                em.name,
                COUNT(industry_id) cnt_industry
            FROM 
                public.employers_industries em_i
                JOIN public.industries ind ON em_i.industry_id = ind.id
                JOIN public.employers em ON em_i.employer_id = em.id
            GROUP BY 1
            HAVING COUNT(industry_id) = 4
            ORDER BY em.name
            '''

In [31]:
exit_5_5 = pd.read_sql_query(query_5_5,connection_project)
display(exit_5_5)

,name,cnt_industry
0,101 Интернет,4
1,21vek.by,4
2,2ГИС,4
3,2К,4
4,4 пикселя +,4
...,...,...
1131,ЮРИОН,4
1132,ЮТИП Технологии,4
1133,ЯКласс,4
1134,ЯрНео,4


Работодателей с четырьмя сферами деятельности более 1к, такие компании могут быть слабоизвестны если каждый из бизнесов, который они ведут, не имеет широкого распространения своего продукта. Также организация может работать в сферах B2B и конечные потребители могут не знать о ней.

#### Количество работодателей, у которых сфера деятельности указана «Разработка программного обеспечения».

In [32]:
query_5_6 = f''' 
            SELECT
                COUNT(em.id)
            FROM 
                public.employers_industries em_i
                JOIN public.industries ind ON em_i.industry_id = ind.id
                JOIN public.employers em ON em_i.employer_id = em.id
            WHERE ind.name LIKE '%Разработка программного обеспечения%'
            '''

In [33]:
exit_5_6 = pd.read_sql_query(query_5_6,connection_project)
display(exit_5_6) 

,count
0,3553


Разработкой ПО занимаются 3,5к организаций, довольно большое число организаций конкурирует в этой среде. Это может говорить об развитии рынка IT, поскольку многие компании и население стараются оптимизировать свою работу через компьютер. Соответственно будет повышаться спрос на специалистов IT, что приведет к увеличнию вакансий в этой области.

#### Для компании «Яндекс» вывести список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавить строку Total с общим количеством вакансий компании.

In [34]:
list_city = ('Москва','Санкт-Петербург',
'Новосибирск','Екатеринбург',
'Казань','Нижний Новгород',
'Челябинск','Красноярск',
'Самара','Уфа',
'Ростов-на-Дону','Омск',
'Краснодар','Воронеж',
'Пермь','Волгоград')

query_5_7 = f''' 
            SELECT 
                ar.name,
                COUNT(vc.id)
            
            FROM public.vacancies vc
                JOIN public.employers em ON em.id = vc.employer_id
                JOIN public.areas ar ON ar.id = vc.area_id
            WHERE em.name = 'Яндекс' AND ar.name in {list_city}
            GROUP BY 1
            
            UNION
            
            SELECT 
                'total', COUNT(*)
            FROM public.vacancies vc
                JOIN public.employers em ON em.id = vc.employer_id
                JOIN public.areas ar ON ar.id = vc.area_id
            WHERE em.name = 'Яндекс' AND ar.name in {list_city}
            ORDER BY 2
            '''

In [35]:
exit_5_7 = pd.read_sql_query(query_5_7,connection_project)
display(exit_5_7) 

,name,count
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


Вакансии Яндекса представлены в каждом городе-миллионнике, организация имеет широкую известность и это способствует развитию бренда на разные товары или услуги. 

In [36]:
import requests # Импортируем библиотеку requests
url = 'http://www.statdata.ru/goroda-millionniki-rossii-po-naseleniu' # Определяем адрес страницы
response = requests.get(url)  # Выполняем GET-запрос, создаём объект requests
print(response.text)  # Выводим содержимое атрибута text

from bs4 import BeautifulSoup # Импортируем библиотеку BeautifulSoup
page = BeautifulSoup(response.text, 'html.parser') # Создаём объект BeautifulSoup, указывая параметр html-парсер
j = page.find('tbody').text
for i in j:
    #if i = ''
    print(i)

<!doctype html>
<html lang="ru-RU">
<head>
	<meta charset="UTF-8">
	<meta name="viewport" content="width=device-width, initial-scale=1">
	<link rel="profile" href="https://gmpg.org/xfn/11">

	<meta name='robots' content='index, follow, max-image-preview:large, max-snippet:-1, max-video-preview:-1' />

	<!-- This site is optimized with the Yoast SEO plugin v19.10 - https://yoast.com/wordpress/plugins/seo/ -->
	<title>Города миллионники России 2022, 2021 список. Города миллионеры России РФ таблица, карта - statdata.ru</title>
	<link rel="canonical" href="http://www.statdata.ru/goroda-millionniki-rossii-po-naseleniu" />
	<meta property="og:locale" content="ru_RU" />
	<meta property="og:type" content="article" />
	<meta property="og:title" content="Города миллионники России 2022, 2021 список. Города миллионеры России РФ таблица, карта - statdata.ru" />
	<meta property="og:description" content="Города миллионники России 2022, 2021 список 11.05.2022 На 1 января 2022 годаЧисленность населения

In [37]:
str1 = """Москва	13 010	13,1%	11 504 
2	Санкт-Петербург	5602	14,8%	4880 
3	Новосибирск	1634	10,9%	1474
4	Екатеринбург	1544	14,4%	1350
5	Казань	1309	14,4%	1144
6	Нижний Новгород	1226	−2,0%	1251
7	Челябинск	1190	5,3%	1130
8	Красноярск	1188	21,6%	977
9	Самара	1173	0,3%	1169
10	Уфа	1145	7,8%	1062
11	Ростов-на-Дону	1142	4,9%	1089
12	Омск	1126	−3,4%	1166
13	Краснодар	1099	43,8%	764
14	Воронеж[a]	1058	18,9%	890
15	Пермь	1034	4,3%	991
16	Волгоград"""
g = []
for i in range(11):
	    g.append(str(i))
     
for i in str1:
    if i in g:
        1
    else:
        i

# Предметный анализ

#### Сколько вакансий имеет отношение к данным?

In [38]:
query_6_1 = f''' 
            SELECT 
                COUNT(name)
            FROM public.vacancies
            WHERE lower(name) LIKE '%данн%' OR
                lower(name) LIKE '%data%'
            '''

In [39]:
exit_5_7 = pd.read_sql_query(query_6_1, connection_project)
display(exit_5_7)

,count
0,1771


#### Сколько есть подходящих вакансий для начинающего дата-сайентиста?

In [40]:
query_6_2 = f''' 
            SELECT
                count(*)
            FROM public.vacancies
            WHERE 
            upper(name) NOT LIKE '%HTML%' AND
                (lower(name) LIKE '%data scientist%' OR
                lower(name) LIKE '%data science%' OR
                lower(name) LIKE '%исследователь данных%' OR
                UPPER(name) LIKE '%ML%' OR 
                lower(name) LIKE '%machine learning%' OR 
                lower(name) LIKE '%машинн%обучен%') AND
                
                (lower(name) LIKE '%junior%' OR
                experience LIKE '%Нет опыта%' OR
                employment LIKE '%Стажировка%') 
            '''
            

In [41]:
exit_6_2 = pd.read_sql_query(query_6_2,connection_project)
display(exit_6_2)

,count
0,51


Среди всех вакансий около 50 подходят начинающим специалистам data science. Профессия новая и число вакансий не велико, но спрос будет расти, поскольку информационные технологии помогают организациям собирать огромные объёмы информации, которые необходимо обрабатывать и извлекать из них ценные данные. 

#### Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

In [42]:
query_6_3 = f''' 
            SELECT
                COUNT(*)
            FROM public.vacancies
            WHERE 
               (lower(name) LIKE '%data scientist%' OR
                lower(name) LIKE '%data science%' OR
                lower(name) LIKE '%исследователь данных%' OR
                UPPER(name) LIKE '%ML%' OR 
                lower(name) LIKE '%machine learning%' OR 
                lower(name) LIKE '%машинн%обучен%') AND
                
                upper(name) NOT LIKE '%HTML%' AND
                lower(name) NOT LIKE '%teamlead%' AND
                (lower(key_skills) LIKE '%sql%' OR lower(key_skills) LIKE '%postgre%')
            '''

In [43]:
exit_6_3 = pd.read_sql_query(query_6_3,connection_project)
display(exit_6_3)

,count
0,201


Навык работы с SQL требуется часто в работе саентиста, поскольку в этой профессии большую часть времени может занимать работа с данными - обработка, анализ, очистка. С помощью SQL можно быстро провести анаиз, получить нужную информацию и с помощью систем BI создать дашборд.

#### Вычислить количество вакансий, в которых в качестве ключевого навыка указан Python.

In [44]:
query_6_4 = f'''
            SELECT
                COUNT(*)
            FROM public.vacancies
            WHERE lower(key_skills) LIKE '%python%' AND
               (lower(name) LIKE '%data scientist%' OR
                lower(name) LIKE '%data science%' OR
                lower(name) LIKE '%исследователь данных%' OR
                UPPER(name) LIKE '%ML%' OR 
                lower(name) LIKE '%machine learning%' OR 
                lower(name) LIKE '%машинн%обучен%') AND
                
                upper(name) NOT LIKE '%HTML%' AND
                lower(name) NOT LIKE '%teamlead%' 
               '''

In [45]:
exit_6_4 = pd.read_sql_query(query_6_4,connection_project)
display(exit_6_4)

,count
0,351


Знание языка python требуется больше чем в половине вакансий для саентистов. Это связано с тем, что он является основным для машинного обучения. Довольно простой и в использовании, гибкий, множество библиотек - все это делает python самым востребованым языком в создании искуственного интеллекта.

#### Сколько ключевых навыков в среднем указывают в вакансиях для DS?

In [46]:
query_6_5 = f''' 
            SELECT 
                AVG(
                    LENGTH(key_skills) - LENGTH(REPLACE(key_skills, '\t','')) +1
                    )
            FROM public.vacancies
            WHERE
               (lower(name) LIKE '%data scientist%' OR
                lower(name) LIKE '%data science%' OR
                lower(name) LIKE '%исследователь данных%' OR
                name LIKE '%ML%' OR
                lower(name) LIKE '%machine learning%' OR 
                lower(name) LIKE '%машинн%обучен%') 
                AND 
                name NOT LIKE '%HTML%' AND
                key_skills IS NOT NULL
            '''

In [47]:
exit_6_5 = pd.read_sql_query(query_6_5,connection_project)
display(exit_6_5)

,avg
0,6.406032


In [48]:
query_6_5 = f''' 
            SELECT 
                key_skills
            FROM public.vacancies
            WHERE
               (lower(name) LIKE '%data scientist%' OR
                lower(name) LIKE '%data science%' OR
                lower(name) LIKE '%исследователь данных%' OR
                name LIKE '%ML%' OR
                lower(name) LIKE '%machine learning%' OR 
                lower(name) LIKE '%машинн%обучен%') 
                AND 
                name NOT LIKE '%HTML%' 
            '''
exit_6_5 = pd.read_sql_query(query_6_5,connection_project)

# создаем объет dataframe
ser = pd.DataFrame(exit_6_5) 

# Выбираем значения без None, ищем в каждой записи значения '/t' и считаем навыки
def func(data):
    
    if data is None:
        return 0
    
    str_skill = list(data)
    sum_skill = str_skill.count('\t') 
    return sum_skill +1

series = ser['key_skills'].apply(func)
#создаём новый датафрайм с кол-вом навыков
new_data = pd.DataFrame(series)
display(round(new_data.mean(),2))


key_skills    5.75
dtype: float64

#### Какую зарплату для DS в среднем указывают для каждого типа требуемого опыта.



In [14]:
query_6_6 = f''' 
            WITH with_null AS 
            (SELECT 
            -- Составим столбцец размера оплаты в случаи, если указана только одна граница зп вилки
                experience,
                coalesce(salary_from, 0) + coalesce(salary_to, 0) mean
                         
            FROM public.vacancies
            WHERE 
               (lower(name) LIKE '%data scientist%' OR
                lower(name) LIKE '%data science%' OR
                lower(name) LIKE '%исследователь данных%' OR
                name LIKE '%ML%' OR
                lower(name) LIKE '%machine learning%' OR 
                lower(name) LIKE '%машинн%обучен%') 
                AND 
                name NOT LIKE '%HTML%' AND
                (salary_from IS NOT NULL OR
                salary_to IS NOT NULL) AND
                (salary_from + salary_to) IS NULL
                
            UNION ALL
            
            SELECT 
            -- Получим размер оплаты в случаи, если указаны обе границы зп вилки и
            -- объеденим с предыдущим запросом 
                experience,
                (salary_from + salary_to)/2
                
            FROM public.vacancies
            WHERE 
               (lower(name) LIKE '%data scientist%' OR
                lower(name) LIKE '%data science%' OR
                lower(name) LIKE '%исследователь данных%' OR
                name LIKE '%ML%' OR
                lower(name) LIKE '%machine learning%' OR 
                lower(name) LIKE '%машинн%обучен%') 
                AND 
                name NOT LIKE '%HTML%' AND
                (salary_from IS NOT NULL OR
                salary_to IS NOT NULL) AND
                (salary_from + salary_to) IS NOT NULL
                )
                
            SELECT 
            -- Группируем по столбцу опыта и считаем среднее значение зарплат 
                experience,
                AVG(mean)
            FROM  with_null
            GROUP BY 1
            '''


In [16]:
exit_6_6 = pd.read_sql_query(query_6_6,connection_project)
display(exit_6_6)

connection_project.close()

,experience,avg
0,От 3 до 6 лет,243114.666667
1,От 1 года до 3 лет,139674.750000
2,Нет опыта,74642.857143


# Общий вывод по проекту